In [272]:
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup


from tqdm import tqdm

import pandas as pd
import polars as pl
import pyarrow as pa


from deltalake.writer import write_deltalake


service = Service(executable_path="/snap/bin/chromium.chromedriver")

from selenium.webdriver.chrome.options import Options
options = Options()
#options.add_argument("start-maximized")
#options.add_argument("disable-infobars")
options.add_argument("--disable-extensions")
#options.add_argument('--headless')
options.add_argument("--disable-dev-shm-usage")
#options.add_argument("--no-sandbox")


In [3]:
driver = webdriver.Chrome(service=service)
season = 2022
comp = 111
round =1

url = f'https://www.nrl.com/ladder/?competition={comp}&round={round}&season={season}'    
driver.get(url)
table_elem = driver.find_element(by=By.CLASS_NAME, value="ladder-page__ladder-inner")
ladder_html = table_elem.get_attribute('innerHTML')



In [2]:
def get_ladder(driver, season: int = 2022, round: int = 1, comp: int = 111) -> pd.DataFrame:
    '''
    Get ladder as at end of specified Round.

    year: starting year of competition
    round: round of competitition
    comp: competition (111 = NRL)
    '''
    url = f'https://www.nrl.com/ladder/?competition={comp}&round={round}&season={season}'    
    driver.get(url)
    table_elem = driver.find_element(by=By.CLASS_NAME, value="ladder-page__ladder-inner")
    ladder_html = table_elem.get_attribute('innerHTML')
    # read_html returns list and we're intentionally only giving it one table to read so just grab 0-th to get dataframe
    ladder_df = pd.read_html(ladder_html)[0]
    ladder_df_slim = ladder_df.drop(['Pos', 'Team', 'Next', 'home', 'away', 'form'], axis=1)
    ladder_df_slim.columns = ['Pos', 'team', 'played', 'points', 'wins', 'drawn', 'lost',
                              'byes', 'for', 'against', 'diff.']
    ladder_df_slim.columns = [c.lower() for c in ladder_df_slim.columns]
    return ladder_df_slim


In [4]:
driver = webdriver.Chrome(service=service)
ldf = get_ladder(driver)
driver.quit()


In [5]:
ldf

,pos,team,played,points,wins,drawn,lost,byes,for,against,diff.
0,1,Panthers,1,2,1,0,0,0,28,6,22
1,2,Knights,1,2,1,0,0,0,20,6,14
2,3,Dragons,1,2,1,0,0,0,28,16,12
3,4,Storm,1,2,1,0,0,0,26,16,10
4,5,Broncos,1,2,1,0,0,0,11,4,7
5,6,Raiders,1,2,1,0,0,0,24,19,5
6,7,Eels,1,2,1,0,0,0,32,28,4
7,8,Bulldogs,1,2,1,0,0,0,6,4,2
8,9,Cowboys,1,0,0,0,1,0,4,6,-2
9,10,Titans,1,0,0,0,1,0,28,32,-4


In [67]:
driver = webdriver.Chrome(service=service)




In [150]:
#driver.implicitly_wait(2)
wait = WebDriverWait(driver, 10)
url = f"https://www.nrl.com/draw/?competition={comp}&round={round}&season={season}"
driver.get(url)

games = wait.until(lambda d: d.find_elements(by=By.CLASS_NAME, value="l-grid"))

# 0-th element of list is 'draw' text from top of page, ignore that
games_list = []
for g in games[1:-2]:
    game_dict = {}
    game_dict['season'] = season
    game_dict['round'] = round
    venue = g.find_element(by=By.CLASS_NAME, value="match-venue").text.splitlines()[1]
    date = g.find_element(by=By.CLASS_NAME, value="match-header__title").text
    teams = g.find_elements(by=By.CLASS_NAME, value='match-team')
    game_dict['venue'] = venue
    game_dict['date'] = date
    for team in teams:
        home_away, team_name = team.find_element(by=By.CLASS_NAME, value="match-team__info").text.splitlines()[:2]
        try:
            points = team.find_element(by=By.CLASS_NAME, value="match-team__score").text.splitlines()[1]
        except IndexError:
            points = None
        home_away = home_away.lower().replace(' ', '_')
        game_dict[f'{home_away}_name'] = team_name 
        game_dict[f'{home_away}_points'] = int(points)
    games_list.append(game_dict)    
games_df = pd.DataFrame(games_list, ) 
games_df.columns = ['season', 'round', 'venue', 'date', 'home_team', 'home_team_points', 'away_team', 'away_team_points']
games_df = games_df.assign(date=pd.to_datetime(games_df.date + ' ' + games_df.season.astype(str)).dt.date)

In [169]:
games_df = pl.DataFrame(games_list)
games_df = games_df.with_columns(date_new=pl.col("date").str.replace(r"(\S{1,}) (\d{1,2})(TH)", r"$2"))
games_df = games_df.with_columns(date=pl.col("date_new") + ' ' + pl.col("season").cast(pl.Utf8))
games_df = games_df.with_columns(date_=pl.col("date").str.strptime(pl.Date, fmt=r"%e %B %Y"))



[(datetime.date(2022, 3, 11),)]

In [234]:
def get_round_list(driver, season: int = 2022, comp: int = 111) -> pd.DataFrame:
    '''
    year: starting year of competition
    round: round of competitition
    comp: competition (111 = NRL)

    '''
    # do hacks until things cause me enough headaches, there should be some sort of wait.until() thing here
    driver.implicitly_wait(2)
    url = f"https://www.nrl.com/draw/?competition={comp}&season={season}"
    driver.get(url)
    rounds = driver.find_elements(by=By.CLASS_NAME, value="filter-dropdown-item--round")
    rounds_list = []
    for i, name in enumerate(rounds):
        rounds_list.append(name.find_element(by=By.TAG_NAME, value='div').get_attribute('innerHTML').splitlines()[1].strip())

    rounds_df = pl.DataFrame({"round_name": rounds_list})
    rounds_df = rounds_df.with_columns(season=pl.lit(season),
                                       round_num=pl.col("round_name").str.extract("Round (\w+)", 1).cast(pl.Int16)
                                       )
    return rounds_df 


def get_round_games(driver, season: int = 2022, round: int = 1, comp: int = 111) -> pd.DataFrame:
    #driver.implicitly_wait(2)
    wait = WebDriverWait(driver, 10)
    url = f"https://www.nrl.com/draw/?competition={comp}&round={round}&season={season}"
    driver.get(url)

    games = wait.until(lambda d: d.find_elements(by=By.CLASS_NAME, value="l-grid"))
    
    # 0-th element of list is 'draw' text from top of page, ignore that
    games_list = []
    for g in games[1:-2]:
        game_dict = {}
        game_dict['season'] = season
        game_dict['round'] = round
        venue = g.find_element(by=By.CLASS_NAME, value="match-venue").text.splitlines()[1]
        date = g.find_element(by=By.CLASS_NAME, value="match-header__title").text
        teams = g.find_elements(by=By.CLASS_NAME, value='match-team')
        game_dict['venue'] = venue
        game_dict['date'] = date
        for team in teams:
            home_away, team_name = team.find_element(by=By.CLASS_NAME, value="match-team__info").text.splitlines()[:2]
            try:
                points = team.find_element(by=By.CLASS_NAME, value="match-team__score").text.splitlines()[1]
            except IndexError:
                points = None
            home_away = home_away.lower().replace(' ', '_')
            game_dict[f'{home_away}_name'] = team_name 
            game_dict[f'{home_away}_points'] = points
        games_list.append(game_dict)    
        games_df = pl.DataFrame(games_list)
        games_df = games_df.with_columns(date=pl.col("date").str.replace(r"(\S{1,}) (\d{1,2})(TH|ST|ND|RD)", r"$2"))
        games_df = games_df.with_columns(date=pl.col("date") + ' ' + pl.col("season").cast(pl.Utf8))
        games_df = games_df.with_columns(date=pl.col("date").str.strptime(pl.Date, fmt=r"%d %B %Y"))
        games_df = games_df.with_columns(home_team_points=pl.col("home_team_points").cast(pl.Int16),
                                         away_team_points=pl.col("away_team_points").cast(pl.Int16))
    return games_df

In [235]:
driver = webdriver.Chrome(service=service)
#driver.implicitly_wait(1)
seasons = range(2018, 2024, 1)
seasons_dfs = []
for s in seasons:
    y = get_round_list(driver, season=s)
    seasons_dfs.append(y)
driver.quit()
all_rounds = pl.concat(seasons_dfs)

In [236]:
all_rounds_list = all_rounds.filter(pl.col("round_num").is_not_null()).select("season", "round_num").rows()

In [237]:
driver = webdriver.Chrome(service=service)
round_dfs = []
ladder_dfs = []

for y, r in tqdm(all_rounds_list):
    round_dfs.append(get_round_games(driver, season=int(y), round=int(r)))
driver.quit()

100%|██████████| 147/147 [01:41<00:00,  1.45it/s]


In [227]:
date

'SUNDAY 13TH MARCH'

In [226]:
round_dfs[-2].with_columns(home_team_points=pl.col("home_team_points").cast(pl.Utf8))

season,round,venue,date,home_team_name,home_team_points,away_team_name,away_team_points
i64,i64,str,date,str,str,str,bool
2023,2,"""BlueBet Stadiu...",2023-03-09,"""Panthers""",null,"""Rabbitohs""",null
2023,2,"""CommBank Stadi...",2023-03-10,"""Eels""",null,"""Sharks""",null
2023,2,"""Suncorp Stadiu...",2023-03-10,"""Broncos""",null,"""Cowboys""",null
2023,2,"""Allianz Stadiu...",2023-03-11,"""Roosters""",null,"""Warriors""",null
2023,2,"""Kayo Stadium, ...",2023-03-11,"""Dolphins""",null,"""Raiders""",null
2023,2,"""AAMI Park, Mel...",2023-03-11,"""Storm""",null,"""Bulldogs""",null
2023,2,"""Leichhardt Ova...",2023-03-12,"""Wests Tigers""",null,"""Knights""",null
2023,2,"""Netstrata Jubi...",2023-03-12,"""Dragons""",null,"""Titans""",null


In [282]:
results = pl.concat(round_dfs)


#results = results.reset_index(drop=True)
#results.to_parquet('../data/raw/game_results.parquet')
results.write_parquet("../data/raw/pl_game_results.parquet")


#results = results.with_columns(venue=pl.col("venue").cast(pl.Categorical),
#                               home_team_name=pl.col('home_team_name').cast(pl.Categorical),
#                               away_team_name=pl.col('away_team_name').cast(pl.Categorical))
# current delta python bindings don't support pandas datetime64[ns], I can't see easy way to convert it other than making dates in to strings (eww)
# current delta python bindings don't support polars LargeUtf8 string type ... there is no winning at this game

# try to_arrow() then changing types to a supported string type, what a joke.

results_arrow = results.to_arrow()


cols_list = ['venue', 'home_team_name', 'away_team_name']
new = {}


for c in results_arrow.column_names:
    if c in cols_list:
        new[c] = results_arrow[c].cast('str')
    else:
        new[c] = results_arrow[c]


results_arrow_updated = pa.Table.from_pydict(new)
write_deltalake("../data/raw/game_results.delta", results_arrow_updated)

